# Problem 1: Binary Words and Operations

## Introduction

This problem implements seven core bitwise functions used in the SHA-256 cryptographic hash algorithm, as defined in the FIPS 180-4 Secure Hash Standard. SHA-256 is widely used in security applications like SSL/TLS, Bitcoin, and digital signatures. These functions perform bitwise operations on 32-bit words to create the confusion and diffusion properties essential for cryptographic security. The logical functions (Ch, Maj, Parity) operate on individual bits, while the rotation functions (Sigma and sigma) combine rotations and shifts with XOR operations to ensure each input bit affects many output bits. Together, they make it computationally infeasible to reverse the hash or find collisions.

## Solution Approach

The solution uses Python's numpy library to ensure all operations are performed on 32-bit unsigned integers (np.uint32), as required by the SHA-256 specification. Each function follows the exact mathematical formulas from FIPS 180-4: the logical functions use AND (&), OR (|), XOR (^), and NOT (~) operators, while the rotation functions use helper functions ROTR (circular right rotation) and SHR (logical right shift). The ROTR operation is implemented using bit shifts and OR operations to wrap bits from the right end to the left, while SHR simply shifts bits right and fills with zeros. This approach ensures compatibility with the standard and produces verifiable results against known test vectors.

## Functions to Implement:

1. `Parity(x, y, z)` - XOR operation across three values
2. `Ch(x, y, z)` - Choice function (conditional bit selection)
3. `Maj(x, y, z)` - Majority function (returns majority bit value)
4. `Sigma0(x)` - written as Σ₀²⁵⁶(x) in the standard
5. `Sigma1(x)` - written as Σ₁²⁵⁶(x) in the standard
6. `sigma0(x)` - written as σ₀²⁵⁶(x) in the standard
7. `sigma1(x)` - written as σ₁²⁵⁶(x) in the standard

## References

**FIPS 180-4: Secure Hash Standard**
- National Institute of Standards and Technology (NIST)
- URL: https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf
- Section 4.1.2, Page 10 - SHA-256 Functions

In [28]:
# ----------------------------------------------------------------------------
# PART 1: Parity Function
# ----------------------------------------------------------------------------
import numpy as np
def Parity(x, y, z):
    """

    Parity function from SHA-256 specification.
    
    Returns the bitwise XOR of x, y, and z.
    Formula: x ⊕ y ⊕ z
    
    This function is used in the SHA-1 algorithm but included here
    for completeness of bitwise operations.
     
    Args:
        x, y, z: 32-bit unsigned integers
    Returns:
        np.uint32: Result of x XOR y XOR z
    """
    # Convert all inputs to 32-bit unsigned integers
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)
    # Perform XOR operation on all three values
    return np.uint32(x ^ y ^ z)  
# ----------------------------------------------------------------------------
# PART 2: Choice (Ch) Function
# ----------------------------------------------------------------------------


def Ch(x, y, z):
    """
    Choice function from SHA-256 specification.
    
    For each bit position, chooses bit from y if x is 1,
    otherwise chooses bit from z.
    
    Formula: (x ∧ y) ⊕ (¬x ∧ z)
    
    Used in: SHA-256 compression function (Step 1 of each round)
    
    Args:
        x, y, z: 32-bit unsigned integers
    
    Returns:
        np.uint32: Result of the choice operation
    """
    # Convert all inputs to 32-bit unsigned integers
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)
    # Where x is 1, choose y; where x is 0, choose z
    return np.uint32((x & y) ^ (~x & z))
# ----------------------------------------------------------------------------
# PART 3: Majority (Maj) Function
# ----------------------------------------------------------------------------

def Maj(x, y , z):
    """
     Majority function from SHA-256 specification.
    
    For each bit position, returns 1 if at least two of the
    three corresponding bits in x, y, z are 1.
    
    Formula: (x ∧ y) ⊕ (x ∧ z) ⊕ (y ∧ z)
    
    Used in: SHA-256 compression function (Step 2 of each round)
    
    Args:
        x, y, z: 32-bit unsigned integers
    
    Returns:
        np.uint32: Result of the majority operation
    """
    # Convert all inputs to 32-bit unsigned integers
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)
    # Return 1 where at least 2 of the 3 bits are 1
    return np.uint32((x & y) ^ (x & z) ^ (y & z))

# ============================================================================
# HELPER FUNCTIONS: ROTR (Rotate Right) and SHR (Shift Right)
# ============================================================================

def ROTR(n, x):
    """
    Rotate right (circular right shift) operation.
    
    Bits shifted off the right end wrap around to the left.
    
    Args:
        n: Number of positions to rotate
        x: 32-bit unsigned integer
    
    Returns:
        np.uint32: x rotated right by n positions
    """
    x = np.uint32(x)
    return np.uint32((x >> n) | (x << (32 - n)))

def SHR(n, x):
    """
    Shift right (logical right shift) operation.
    
    Bits shifted off the right are lost, zeros fill from the left.
    
    Args:
        n: Number of positions to shift
        x: 32-bit unsigned integer
    
    Returns:
        np.uint32: x shifted right by n positions
    """
    x = np.uint32(x)
    return np.uint32(x >> n )

# ----------------------------------------------------------------------------
# PART 4: Sigma0 Function (Uppercase Σ₀²⁵⁶)
# ----------------------------------------------------------------------------
def Sigma0(x):
    """
    Sigma0 function from SHA-256 specification.
    
    Combines three rotations of x with XOR operation.
    Formula: ROTR²(x) ⊕ ROTR¹³(x) ⊕ ROTR²²(x)
    
    Used in: SHA-256 compression function for computing temp2 value
    
    Args:
        x: 32-bit unsigned integer
    
    Returns:
        np.uint32: Result of Sigma0 operation
    """
    x = np.uint32(x)
    # XOR of three different rotations
    return np.uint32(ROTR(2, x) ^ ROTR(13, x) ^ ROTR(22, x))

# ----------------------------------------------------------------------------
# PART 5: Sigma1 Function (Uppercase Σ₁²⁵⁶)
# ----------------------------------------------------------------------------

def Sigma1(x):
    """

    Sigma1 function from SHA-256 specification.
    Formula :  ROTR^6(x) XOR ROTR^11(x) XOR ROTR^25(x)

    Used in: SHA-256 compression function for computing temp1 value
    
    Args:
        x: 32-bit unsigned integer
    
    Returns:
        np.uint32: Result of Sigma1 operation
    """
    x = np.uint32(x)
    # XOR of three different rotations
    return np.uint32(ROTR(6, x) ^ ROTR(11, x) ^ ROTR(25, x))


# ----------------------------------------------------------------------------
# PART 6: sigma0 Function (Lowercase σ₀²⁵⁶)
# ----------------------------------------------------------------------------
def sigma0(x):
    """
    sigma0 (lowercase) function from SHA-256 specification.
    
    Combines two rotations and one shift with XOR operation.
    Formula: ROTR^7(x) XOR ROTR^18(x) XOR SHR^3(x)
    
    Note: Uses SHR (shift) for the third operation, not ROTR (rotate)
    
    Used in: SHA-256 message schedule expansion (extending 16 words to 64 words)
    
    Args:
        x: 32-bit unsigned integer
    
    Returns:
        np.uint32: Result of sigma0 operation
    """
    x = np.uint32(x)
    # XOR of two rotations and one shift
    return np.uint32(ROTR(7, x) ^ ROTR(18, x) ^ SHR(3, x))

# ----------------------------------------------------------------------------
# PART 7: sigma1 Function (Lowercase σ₁²⁵⁶)
# ----------------------------------------------------------------------------
def sigma1(x):
    """
    sigma1 (lowercase) function from SHA-256 specification.
    
    Combines two rotations and one shift with XOR operation.
    Formula: ROTR^17(x) XOR ROTR^19(x) XOR SHR^10(x)
    
    Note: Uses SHR (shift) for the third operation, not ROTR (rotate)
    
    Used in: SHA-256 message schedule expansion (extending 16 words to 64 words)
    
    Args:
        x: 32-bit unsigned integer
    
    Returns:
        np.uint32: Result of sigma1 operation
    """
    x = np.uint32(x)
    # XOR of two rotations and one shift
    return np.uint32(ROTR(17, x) ^ ROTR(19, x) ^ SHR(10, x))



In [29]:
# Test 1 the Parity function
x = np.uint32(0b1100)
y = np.uint32(0b1010)
z = np.uint32(0b0110)

result = Parity(x, y, z)
print(f"Parity({x}, {y}, {z}) = {result}")
print(f"Binary: {bin(result)}")


#Test 1.2: All zeros
x, y, z = np.uint32(0), np.uint32(0), np.uint32(0)
result = Parity(x, y, z)
print(f"Test 1.2: Parity(0, 0, 0) = {result} (Expected: 0)")

# Test 1.3: All ones
x = np.uint32(0xFFFFFFFF)
y = np.uint32(0xFFFFFFFF)
z = np.uint32(0xFFFFFFFF)
result = Parity(x, y, z)
print(f"Test 1.3: Parity(all 1s) = {hex(result)} (Expected: 0xffffffff)")



Parity(12, 10, 6) = 0
Binary: 0b0
Test 1.2: Parity(0, 0, 0) = 0 (Expected: 0)
Test 1.3: Parity(all 1s) = 0xffffffff (Expected: 0xffffffff)


In [30]:
# 2.Test the Ch function
x = np.uint32(0b1100)
y = np.uint32(0b1010)
z = np.uint32(0b0110)

result = Ch(x, y, z)
print(f"Ch({x}, {y}, {z}) = {result}")
print(f"Binary: {bin(result)}")

# Test 2.2 : x all ones (chooses y)
x = np.uint32(0xFFFFFFFF)
y = np.uint32(0x12345678)
z = np.uint32(0x9ABCDEF0)
result = Ch(x, y, z)
print(f"Test 2.2: Ch(all 1s, y, z) = {hex(result)} (Expected: 0x12345678)")

# Test 2.3: x all zeros (chooses z)
x = np.uint32(0)
y = np.uint32(0x12345678)
z = np.uint32(0x9ABCDEF0)
result = Ch(x, y, z)
print(f"Test 2.3: Ch(all 0s, y, z) = {hex(result)} (Expected: 0x9abcdef0)")

print("\n" + "=" * 70)




Ch(12, 10, 6) = 10
Binary: 0b1010
Test 2.2: Ch(all 1s, y, z) = 0x12345678 (Expected: 0x12345678)
Test 2.3: Ch(all 0s, y, z) = 0x9abcdef0 (Expected: 0x9abcdef0)



In [31]:
# ----------------------------------------------------------------------------
# PART 3: Maj (Majority) Function Tests
# ----------------------------------------------------------------------------
print("\n--- PART 3: Maj (Majority) Function Tests ---")

# Test 3.1: Basic test
x = np.uint32(0b1100)
y = np.uint32(0b1010)
z = np.uint32(0b0110)
result = Maj(x, y, z)
print(f"Test 3.1: Maj({x}, {y}, {z}) = {result}")
print(f"         Binary: {bin(result)}")

# Test 3.2: All zeros
x, y, z = np.uint32(0), np.uint32(0), np.uint32(0)
result = Maj(x, y, z)
print(f"Test 3.2: Maj(0, 0, 0) = {result} (Expected: 0)")

# Test 3.3: All ones
x = np.uint32(0xFFFFFFFF)
y = np.uint32(0xFFFFFFFF)
z = np.uint32(0xFFFFFFFF)
result = Maj(x, y, z)
print(f"Test 3.3: Maj(all 1s) = {hex(result)} (Expected: 0xffffffff)")

# Test 3.4: Two agree on 1s (majority wins)
x = np.uint32(0xFFFFFFFF)
y = np.uint32(0xFFFFFFFF)
z = np.uint32(0x00000000)
result = Maj(x, y, z)
print(f"Test 3.4: Maj(1s, 1s, 0s) = {hex(result)} (Expected: 0xffffffff)")

print("\n" + "=" * 70)



--- PART 3: Maj (Majority) Function Tests ---
Test 3.1: Maj(12, 10, 6) = 14
         Binary: 0b1110
Test 3.2: Maj(0, 0, 0) = 0 (Expected: 0)
Test 3.3: Maj(all 1s) = 0xffffffff (Expected: 0xffffffff)
Test 3.4: Maj(1s, 1s, 0s) = 0xffffffff (Expected: 0xffffffff)



In [32]:
# ----------------------------------------------------------------------------
# PART 4: Sigma0 Function Tests
# ----------------------------------------------------------------------------
print("\n--- PART 4: Sigma0 (Uppercase Σ₀) Function Tests ---")

# Test 4.1: Basic test
x = np.uint32(0x12345678)
result = Sigma0(x)
print(f"Test 4.1: Sigma0(0x12345678) = {hex(result)}")

# Test 4.2: All zeros
x = np.uint32(0)
result = Sigma0(x)
print(f"Test 4.2: Sigma0(0) = {result} (Expected: 0)")

# Test 4.3: All ones
x = np.uint32(0xFFFFFFFF)
result = Sigma0(x)
print(f"Test 4.3: Sigma0(all 1s) = {hex(result)} (Expected: 0xffffffff)")

# Test 4.4: Alternating bits
x = np.uint32(0xAAAAAAAA)
result = Sigma0(x)
print(f"Test 4.4: Sigma0(0xAAAAAAAA) = {hex(result)}")


--- PART 4: Sigma0 (Uppercase Σ₀) Function Tests ---
Test 4.1: Sigma0(0x12345678) = 0x66146474
Test 4.2: Sigma0(0) = 0 (Expected: 0)
Test 4.3: Sigma0(all 1s) = 0xffffffff (Expected: 0xffffffff)
Test 4.4: Sigma0(0xAAAAAAAA) = 0x55555555


In [33]:
# ----------------------------------------------------------------------------
# PART 5: Sigma1 Function Tests
# ----------------------------------------------------------------------------
print("\n--- PART 5: Sigma1 (Uppercase Σ₁) Function Tests ---")

# Test 5.1: Basic test
x = np.uint32(0x12345678)
result = Sigma1(x)
print(f"Test 5.1: Sigma1(0x12345678) = {hex(result)}")

# Test 5.2: All zeros
x = np.uint32(0)
result = Sigma1(x)
print(f"Test 5.2: Sigma1(0) = {result} (Expected: 0)")

# Test 5.3: All ones
x = np.uint32(0xFFFFFFFF)
result = Sigma1(x)
print(f"Test 5.3: Sigma1(all 1s) = {hex(result)} (Expected: 0xffffffff)")

# Test 5.4: Alternating bits
x = np.uint32(0xAAAAAAAA)
result = Sigma1(x)
print(f"Test 5.4: Sigma1(0xAAAAAAAA) = {hex(result)}")


--- PART 5: Sigma1 (Uppercase Σ₁) Function Tests ---
Test 5.1: Sigma1(0x12345678) = 0x3561abda
Test 5.2: Sigma1(0) = 0 (Expected: 0)
Test 5.3: Sigma1(all 1s) = 0xffffffff (Expected: 0xffffffff)
Test 5.4: Sigma1(0xAAAAAAAA) = 0xaaaaaaaa


In [34]:
# ----------------------------------------------------------------------------
# PART 6: sigma0 Function Tests
# ----------------------------------------------------------------------------
print("\n--- PART 6: sigma0 (Lowercase σ₀) Function Tests ---")

# Test 6.1: Basic test
x = np.uint32(0x12345678)
result = sigma0(x)
print(f"Test 6.1: sigma0(0x12345678) = {hex(result)}")

# Test 6.2: All zeros
x = np.uint32(0)
result = sigma0(x)
print(f"Test 6.2: sigma0(0) = {result} (Expected: 0)")

# Test 6.3: All ones
x = np.uint32(0xFFFFFFFF)
result = sigma0(x)
print(f"Test 6.3: sigma0(all 1s) = {hex(result)}")

# Test 6.4: Power of 2
x = np.uint32(0x80000000)
result = sigma0(x)
print(f"Test 6.4: sigma0(0x80000000) = {hex(result)}")


--- PART 6: sigma0 (Lowercase σ₀) Function Tests ---
Test 6.1: sigma0(0x12345678) = 0xe7fce6ee
Test 6.2: sigma0(0) = 0 (Expected: 0)
Test 6.3: sigma0(all 1s) = 0x1fffffff
Test 6.4: sigma0(0x80000000) = 0x11002000


In [35]:
# ----------------------------------------------------------------------------
# PART 7: sigma1 Function Tests
# ----------------------------------------------------------------------------
print("\n--- PART 7: sigma1 (Lowercase σ₁) Function Tests ---")

# Test 7.1: Basic test
x = np.uint32(0x12345678)
result = sigma1(x)
print(f"Test 7.1: sigma1(0x12345678) = {hex(result)}")

# Test 7.2: All zeros
x = np.uint32(0)
result = sigma1(x)
print(f"Test 7.2: sigma1(0) = {result} (Expected: 0)")

# Test 7.3: All ones
x = np.uint32(0xFFFFFFFF)
result = sigma1(x)
print(f"Test 7.3: sigma1(all 1s) = {hex(result)}")

# Test 7.4: Power of 2
x = np.uint32(0x80000000)
result = sigma1(x)
print(f"Test 7.4: sigma1(0x80000000) = {hex(result)}")

print("\n" + "=" * 70)
print("ALL PROBLEM 1 FUNCTIONS COMPLETE!")
print("=" * 70)


--- PART 7: sigma1 (Lowercase σ₁) Function Tests ---
Test 7.1: sigma1(0x12345678) = 0xa1f78649
Test 7.2: sigma1(0) = 0 (Expected: 0)
Test 7.3: sigma1(all 1s) = 0x3fffff
Test 7.4: sigma1(0x80000000) = 0x205000

ALL PROBLEM 1 FUNCTIONS COMPLETE!


## Conclusion

This problem successfully implemented all seven bitwise functions required for the SHA-256 algorithm as specified in FIPS 180-4. The logical functions (Parity, Ch, Maj) provide non-linear bit mixing through conditional operations and majority voting, while the rotation functions (Sigma0, Sigma1, sigma0, sigma1) create diffusion by combining multiple rotations and shifts with XOR operations. All functions were tested with multiple test cases including edge cases (all zeros, all ones, and specific bit patterns) to verify correctness against the specification. The implementation uses numpy's uint32 type to ensure proper 32-bit arithmetic and modulo 2^32 behavior. These seven functions form the cryptographic foundation for the complete SHA-256 hash algorithm that will be implemented in subsequent problems.

## Problem 2: Fractional Parts of Cube Roots

### Introduction

This problem calculates the constants used in the SHA-256 message schedule, specifically the round constants K. These constants are derived from the first 32 bits of the fractional parts of the cube roots of the first 64 prime numbers. Using mathematical constants like these ensures that the values are "nothing up my sleeve" numbers - they're derived from a transparent mathematical process rather than arbitrary choices, which is important for cryptographic security and trust in the algorithm.

### Solution Approach

The solution implements a prime number generator using trial division, calculates cube roots using numpy's cbrt function, extracts the fractional part by subtracting the integer part, and then converts the fractional part to a 32-bit hexadecimal representation by multiplying by 2^32. The results are verified against the constants listed in FIPS 180-4 (page 11) to ensure correctness.

### References

**FIPS 180-4: Secure Hash Standard**
- National Institute of Standards and Technology (NIST)
- URL: https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf
- Section 4.2.2, Page 11 - SHA-256 Constants
- Relevance: Official specification listing the 64 round constants derived from cube roots of the first 64 primes.

**Wikipedia - Nothing-up-my-sleeve number**
- URL: https://en.wikipedia.org/wiki/Nothing-up-my-sleeve_number
- Relevance: Explains why cryptographic algorithms use mathematically derived constants rather than arbitrary values to prevent hidden backdoors and increase trust.

**Wikipedia - Prime number**
- URL: https://en.wikipedia.org/wiki/Prime_number
- Relevance: Provides background on prime number properties and generation algorithms used in this implementation.

In [36]:
# ============================================================================
# PROBLEM 2: Fractional Parts of Cube Roots
# ============================================================================

import numpy as np

# ----------------------------------------------------------------------------
# STEP 1: Prime Number Generator
# ----------------------------------------------------------------------------

def primes(n):
    """
    Generate the first n prime numbers.
    
    Uses trial division method: checks if each candidate number
    is divisible by any previously found primes.
    
    Args:
        n: Number of primes to generate
    
    Returns:
        list: List of the first n prime numbers
    
    Example:
        >>> primes(5)
        [2, 3, 5, 7, 11]
    """
    if n <= 0:
        return []
    
    prime_list = []  # Store found primes
    candidate = 2    # Start checking from 2 (first prime)
    
    while len(prime_list) < n:
        is_prime = True
        
        # Check if candidate is divisible by any existing prime
        for prime in prime_list:
            if candidate % prime == 0:
                is_prime = False
                break
            # Optimization: only check up to sqrt(candidate)
            if prime * prime > candidate:
                break
        
        if is_prime:
            prime_list.append(candidate)
        
        candidate += 1
    
    return prime_list


# ----------------------------------------------------------------------------
# Test the Prime Generator
# ----------------------------------------------------------------------------
print("=" * 70)
print("PROBLEM 2: Testing Prime Number Generator")
print("=" * 70)

# Test with first 10 primes
first_10 = primes(10)
print(f"\nFirst 10 primes: {first_10}")
print(f"Expected: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]")

# Test with first 20 primes
first_20 = primes(20)
print(f"\nFirst 20 primes: {first_20}")

# Generate the 64 primes we need for SHA-256
first_64 = primes(64)
print(f"\nFirst 64 primes generated successfully!")
print(f"First prime: {first_64[0]}, Last prime: {first_64[-1]}")

print("=" * 70)

# ----------------------------------------------------------------------------
# STEP 2: Calculate Cube Roots and Extract Fractional Parts
# ----------------------------------------------------------------------------

# Get the first 64 primes
primes_64 = primes(64)

print("\n" + "=" * 70)
print("Calculating Cube Roots of First 64 Primes")
print("=" * 70)

# Calculate cube roots and extract fractional parts
cube_roots = []
fractional_parts = []

for i, prime in enumerate(primes_64):
    # Calculate cube root
    cbrt = np.cbrt(prime)
    
    # Extract fractional part (remove integer part)
    frac = cbrt - int(cbrt)
    
    cube_roots.append(cbrt)
    fractional_parts.append(frac)
    
    # Display first 5 as examples
    if i < 5:
        print(f"Prime {prime:3d}: cbrt = {cbrt:.10f}, fractional part = {frac:.10f}")

print(f"\n... (calculated for all 64 primes)")
print(f"Total fractional parts calculated: {len(fractional_parts)}")
print("=" * 70)

# ----------------------------------------------------------------------------
# STEP 3: Convert Fractional Parts to 32-bit Hexadecimal
# ----------------------------------------------------------------------------

print("\n" + "=" * 70)
print("Converting Fractional Parts to 32-bit Hexadecimal Constants")
print("=" * 70)

# Convert fractional parts to 32-bit hex values
hex_constants = []

for i, frac in enumerate(fractional_parts):
    # Multiply by 2^32 to get the first 32 bits
    # Convert to unsigned 32-bit integer
    value = np.uint32(frac * (2**32))
    hex_constants.append(value)
    
    # Display first 10 as examples
    if i < 10:
        print(f"K[{i:2d}] = 0x{value:08x}  (prime = {primes_64[i]:3d})")

print(f"\n... (calculated for all 64 primes)")
print(f"\nTotal constants generated: {len(hex_constants)}")

# Display all 64 constants in a formatted table
print("\n" + "=" * 70)
print("All 64 SHA-256 Round Constants (K)")
print("=" * 70)

for i in range(0, 64, 4):
    # Print 4 constants per line
    line = "  ".join([f"0x{hex_constants[j]:08x}" for j in range(i, min(i+4, 64))])
    print(f"K[{i:2d}-{min(i+3, 63):2d}]: {line}")

print("=" * 70)




PROBLEM 2: Testing Prime Number Generator

First 10 primes: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
Expected: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]

First 20 primes: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71]

First 64 primes generated successfully!
First prime: 2, Last prime: 311

Calculating Cube Roots of First 64 Primes
Prime   2: cbrt = 1.2599210499, fractional part = 0.2599210499
Prime   3: cbrt = 1.4422495703, fractional part = 0.4422495703
Prime   5: cbrt = 1.7099759467, fractional part = 0.7099759467
Prime   7: cbrt = 1.9129311828, fractional part = 0.9129311828
Prime  11: cbrt = 2.2239800906, fractional part = 0.2239800906

... (calculated for all 64 primes)
Total fractional parts calculated: 64

Converting Fractional Parts to 32-bit Hexadecimal Constants
K[ 0] = 0x428a2f98  (prime =   2)
K[ 1] = 0x71374491  (prime =   3)
K[ 2] = 0xb5c0fbcf  (prime =   5)
K[ 3] = 0xe9b5dba5  (prime =   7)
K[ 4] = 0x3956c25b  (prime =  11)
K[ 5] = 0x59f111f1  (prim